In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import pandas as pd
import numpy as np
import catboost as cb 
import seaborn as sns
import matplotlib.pyplot as plt
import re
#import sharp 
from sklearn.impute import SimpleImputer
from sklearn.metrics import auc
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split 
from pandas_profiling import ProfileReport
from catboost import CatBoostRegressor
# Make necessary imports
from catboost import CatBoostRegressor, Pool, cv
from sklearn.metrics import accuracy_score
#To show plots
from ipywidgets import interact  
import ipywidgets as widgets  
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold,train_test_split, RepeatedKFold
from sklearn.metrics import mean_squared_error as mse
import xgboost as xgb

# Suppress the warning outputs

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Set the output displaying: max number of rows & max number of columns 

In [31]:
pd.set_option('display.max_columns', 100)  # or 1000
pd.set_option('display.max_rows', 100)  # or 1000

# Import the data, train and test 

In [26]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test= pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

## Look at basic informations about the train and test data: dimensions and columns 

In [10]:
print(test.shape)
print(train.shape)
print(train.columns)
print(test.equals(train)) ## check if the columns are the same 
print([i for i in train.columns if i not in test.columns]) # find out that only SalePrice is not in test

In [11]:
train.head(5) # look at the details of the data, including numerical and char columns

## Histogram of the output variable and take the log transformation to see the distributions

In [12]:
#sns.set(rc={'figure.figsize':(9,3)})
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16,5))
s = sns.distplot((train['SalePrice']),ax = ax1)
s = sns.distplot(np.log(train['SalePrice']), ax = ax2)
plt.suptitle("Price")
plt.show(); ## after taking the log, it is more normally distributed 

## Look at missing data and some variable distribution

In [27]:
#train.isnull().sum() * 100 / len(train)
# Exploring missing data and checking if any has NaN values
plt.figure(figsize=(7, 5))
sns.heatmap(train.isnull(), cmap='viridis') # some variable have many missing values, need to check those variables
#train.isnull().any() 

In [34]:
train.isnull().sum().sort_values(ascending =False)[:25] ## 19 variables contain missing values 

## After looking at those missing variables in detail, 
### take some variables as the examples: 
### some missing values are actually meaning 0, so do the following imputations:

In [21]:
train['Alley'] = train['Alley'].replace(np.nan, "NoAlley")
sns.boxplot( x="Alley", y="SalePrice", data=train, palette="PRGn",showfliers=True)
## after impute the missing by "NoAlley", look at the boxplot, it seems no very obvious distinct range 
## between NoAlley and other groups, but NoAlley has a lot outliers, which also might because most of 
## the values are NoAlley

In [35]:
train[train['LotFrontage'].isnull()]
pd.crosstab(train.LotFrontage, train.Street, dropna=False)
sns.barplot(x="Street", y="SalePrice", data=train) ## look at the difference of saleprice between two groups of LotFrontage

In [36]:
pd.crosstab(train.GarageType, train.GarageCars).plot(kind="bar")
## check if clear trend between garagetype and garagecars, seems Attached and Dettaced has more observations, 
## most of the families have 1-2 cars 

# After checking all missing varaibles 
# handle missing values and also generate some extra potential variables

In [37]:
for i in ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Electrical','MasVnrType']:
    train[i].fillna('None',inplace=True)
    test[i].fillna('None',inplace=True)

garages = ['GarageType','GarageFinish','GarageQual','GarageCond']
train[garages]= train[garages].replace(np.nan, "NoGarage")
test[garages]= test[garages].replace(np.nan, "NoGarage")
train['MasVnrArea'].fillna(0, inplace=True)
test['MasVnrArea'].fillna(0, inplace=True)
train['LotFrontage'].fillna(np.mean(train['LotFrontage']), inplace=True)
test['LotFrontage'].fillna(np.mean(test['LotFrontage']), inplace=True)


train = train.drop(["PoolQC","Fence","MiscFeature",'GarageYrBlt','Id'], axis=1)   
test = test.drop(["PoolQC","Fence","MiscFeature",'GarageYrBlt','Id'], axis=1)   

train['Alley'] = train['Alley'].replace(np.nan, "NoAlley")
test['Alley'] = test['Alley'].replace(np.nan, "NoAlley")
train['FireplaceQu'] = train['FireplaceQu'].replace(np.nan, "NoFireplace")
test['FireplaceQu'] = test['FireplaceQu'].replace(np.nan, "NoFireplace")

# Generate some new variables, such as house age, total house are, total number of rooms etc

In [38]:
train['houseage1'] = train['YrSold'] - train['YearBuilt']+1
train['houseage2'] = train['YrSold'] - train['YearRemodAdd']+1
train['total_area'] = np.log1p(train['GrLivArea'] + train['TotalBsmtSF'] + train['1stFlrSF'] + train['2ndFlrSF'])
train['num_rooms'] = train['TotRmsAbvGrd'] + train['BedroomAbvGr'] + train['FullBath']
test['houseage1'] = test['YrSold'] - test['YearBuilt']+1
test['houseage2'] = test['YrSold'] - test['YearRemodAdd']+1
test['total_area'] = np.log1p(test['GrLivArea'] + test['TotalBsmtSF'] + test['1stFlrSF'] + test['2ndFlrSF'])
test['num_rooms'] = test['TotRmsAbvGrd'] + test['BedroomAbvGr'] + test['FullBath']


In [39]:
train['bath_rooms'] = train['HalfBath'] + train['FullBath']
test['bath_rooms'] = test['HalfBath'] + test['FullBath']

In [40]:
train['logsaleprice'] = np.log(train['SalePrice'])
train = train.drop(['SalePrice'], axis=1)   

In [41]:
train['totalsf'] = train["1stFlrSF"] + train["2ndFlrSF"] + train["TotalBsmtSF"] 
test['totalsf'] = test["1stFlrSF"] + test["2ndFlrSF"] + test["TotalBsmtSF"] 
train['otherarea'] = train["LotArea"] - train["GrLivArea"] + train["GarageArea"] 
test['otherarea'] = test["LotArea"]- test["GrLivArea"] + test["GarageArea"] 

# Look at categorical variables

In [42]:
cols=train.select_dtypes(exclude='number').columns.to_list()
cat=train[cols].astype('category') ## all categorical variables
cata_price=pd.concat([train[cat.columns.to_list()], train['logsaleprice']], axis=1)

In [43]:
#  Categorical Data
a = 10  # number of rows
b = 4  # number of columns
c = 1  # initialize plot counter

fig = plt.figure(figsize=(20,40))

for column in cata_price.columns[:-1]:
    plt.subplot(a, b, c)
    plt.subplots_adjust(hspace=0.3)
    plt.title('{}, subplot: {}{}{}'.format(column, a, b, c))
    plt.xlabel(column)
    sns.boxplot(x=column, y="logsaleprice", data=cata_price)
    c = c + 1

plt.show()
## look at the boxplots, and we can see some variables have similar patterns on the log price,such as 
## KitchenQual vs GarageFinish; GarageQual vs SaleCondition; MasVnrType vs ExterQual 

# Look at numerical variables 

In [44]:
nums = train.select_dtypes(include='number')
vals= [i for i in nums.columns if i!='logsaleprice'] 

In [45]:
#  plot Numerical Data
a = 7  # number of rows
b = 6  # number of columns
c = 1  # initialize plot counter

fig = plt.figure(figsize=(20,15))

for i in vals:
    plt.subplot(a, b, c)
    plt.subplots_adjust(hspace=0.8)
    plt.title('{} (dist), subplot: {}{}{}'.format(i, a, b, c))
    plt.xlabel(i)
    sns.scatterplot(data=train, x=train[i], y="logsaleprice")
    c = c + 1

plt.show()
## some numerical variables have clear increasing trend when price is up, such as GrLivArea, YearBuild, 
## total area might have some polynomial trend but not only linear

# Checking outliers 
## I didn't remove the outliers here, because those outliers might also provide some information 
## For example, some luxury house with very large LotArea might have extrem high price 

In [46]:
#checking for outliers
plt.figure(figsize=(22,10))
plt.xticks(rotation =90)
sns.boxplot(data = train[vals])
plt.show()

# Checking corrections between all those numerical variables 
## Briefly look at the linear relationship between numberical variables

In [102]:
# Looking at the correlation table, only showing part of them, otherwise plot is too large
plt.figure(figsize = (10,10))
#print(train[vals].iloc[:,0:10])
sns.heatmap(train[vals].iloc[:,10:20].corr(),cmap="YlGnBu", annot=True)
plt.show() ## seems 1stFlrSF btw BsmtUnfSF has high correlation = 0.82, GrLivAre btw 2ndFlrSF has corrlation = 0.69
## however the tree algorithm are noneparametric modeling, they can handle high correlation better than regression

# Convert categorical to dummy variables 

In [63]:
cols_train=train.select_dtypes(exclude='number').columns.to_list()
cat_train=train[cols_train].astype('category')
train1= pd.get_dummies(data=train, columns=cat_train.columns, prefix='cat', prefix_sep='_')
train1 = train1.loc[:,~train1.columns.duplicated()]

In [64]:
cols_test=test.select_dtypes(exclude='number').columns.to_list()
cat_test=test[cols_test].astype('category')
test1= pd.get_dummies(data=test, columns=cat_test.columns, prefix='cat', prefix_sep='_')
test1 = test1.loc[:,~test1.columns.duplicated()]

In [65]:
print([i for i in train1.columns if i not in test1.columns]) 

# Drop those variables 

In [66]:
train1 = train1.drop(['cat_NoSeWa', 'cat_2.5Fin', 'cat_ClyTile', 'cat_Membran', 'cat_Metal', 'cat_Roll', 'cat_Other', 'cat_Floor', 'cat_OthW', 'cat_Mix'],axis=1)
train1.shape

In [67]:
test1.shape

# Split train dataset into train vs validation 

In [68]:
X = train1.loc[:, train1.columns!='logsaleprice']  
y = train1['logsaleprice']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 10)

# Cat boost model 

## model parameter tuning 

In [69]:
def rmsle(y_log, y0_log):
    return np.sqrt(np.mean(np.square(y_log - y0_log)))

def catboost(X_train,y_train,X_val, y_val):
    param = { 'iterations': 3000,
            'od_type':'Iter',
            'learning_rate':0.05,
            'depth':5,
            'bagging_temperature':0.2,
             'eval_metric':'RMSE',
             'random_seed': 1234,
            'metric_period':50,
            'od_wait': 50}

    param_test1 = {
     'depth':[5],
     'bagging_temperature':[0.2,0.3],
     'learning_rate':[0.05,0.1],
      }

    gsearch = GridSearchCV(CatBoostRegressor(learning_rate = param['learning_rate'], 
                                                 depth=param['depth'],
                                                 eval_metric=param['eval_metric'],
                                                 random_seed = param['random_seed'],
                                                 bagging_temperature = param['bagging_temperature'],
                                                 od_type= param['od_type'],
                                                 metric_period = param['metric_period'],
                                                 od_wait=param['od_wait']
                                                ),
                            param_grid = param_test1, scoring='neg_mean_squared_error',iid=False, cv=5)
    
    gsearch.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, verbose=50)
    print(gsearch.cv_results_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
    return gsearch.best_estimator_

## using the tuned paramters

In [70]:
cb_model = CatBoostRegressor(iterations=500,
                             learning_rate=0.05,
                             depth=5,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)
cbevaluate=cb_model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, verbose=50, plot=True)

In [71]:
cbresult= cbevaluate.get_evals_result()

## Plot the RMSE vs iterations, we see that the RMSE is decresing to ~ 0.125 on the validation set 

In [72]:
plt.plot(list(range(0, 348)), cbresult['validation']['RMSE'])

In [73]:
cbevaluate.get_best_score()

## The feature importance plot showed us the top important variables 

In [74]:
tmp_cat = pd.DataFrame({'Feature': X_train.columns, 'Feature importance': cb_model.feature_importances_})
tmp_cat = tmp_cat.sort_values(by='Feature importance',ascending=False)
plt.figure(figsize = (20,8))
plt.title('Features importance',fontsize=14)
s = sns.barplot(x='Feature',y='Feature importance',data=tmp_cat[:30])
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.show()  

In [76]:
tmp_cat[:20]

In [77]:
validatey=cb_model.predict(X_val)
rmsle(y_val, validatey)

# Xgboost model 

# the parameters has already by tuned 

In [78]:
xgbmodel= xgb.XGBRegressor(learning_rate=0.05, 
                           max_depth=5, 
                           min_child_weight=1, 
                           n_estimators=500, 
                           random_state=1234,
                           alpha=0.01, 
                           subsample=0.8,
                           colsample_bytree=0.6,
                           tree_method='exact', 
                           eval_metric='rmse')

In [79]:
#xgbmodel.fit(X_train, y_train)
#model.fit(train_X, train_y, early_stopping_rounds=50, eval_set=[(test_X, test_y)], verbose=True)
xgbmodelfit=xgbmodel.fit(X_train, y_train, verbose=True, eval_set=[(X_val, y_val)])

In [80]:
evals_result = xgbmodelfit.evals_result()
tempresulst = evals_result['validation_0']['rmse']
print(len(tempresulst))
plt.plot(list(range(0, 500)), tempresulst)

# The best RMSE is 0.132

In [81]:
xgb_valid=xgbmodel.predict(X_val)
rmsle(y_val, xgb_valid)

In [82]:
tmp_xgboost = pd.DataFrame({'Feature': X_train.columns, 'Feature importance': xgbmodelfit.feature_importances_})
tmp_xgboost = tmp_xgboost.sort_values(by='Feature importance',ascending=False)
plt.figure(figsize = (20,8))
plt.title('Features importance',fontsize=14)
s = sns.barplot(x='Feature',y='Feature importance',data=tmp_xgboost[:30])
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.show()  

In [83]:
tmp_xgboost[:20]

## Random Forest

In [84]:
model = RandomForestRegressor(n_jobs=-1,random_state =1234)
# Try different numbers of n_estimators - takes some time
estimators = np.arange(1, 500, 10)
rmse_rf = []
for n in estimators:
    model.set_params(n_estimators=n)
    model.fit(X_train, y_train)
    ypred = model.predict(X_val)
    rmse_rf.append(np.sqrt(np.mean(np.square(y_val - ypred))))  


In [88]:
plt.figure(figsize=(7, 5))
plt.title("Effect of Estimators")
plt.xlabel("no. estimator")
plt.ylabel("RMSE for Randomforest")
plt.ylim((0.1,0.25))
plt.plot(estimators, rmse_rf)

In [89]:
%timeit
rf_model = RandomForestRegressor(n_estimators = 500, n_jobs = -1,random_state =1234)
rfmodel = rf_model.fit(X_train, y_train)

In [90]:
tmp_rf = pd.DataFrame({'Feature': X_train.columns, 'Feature importance': rfmodel.feature_importances_})
tmp_rf = tmp_rf.sort_values(by='Feature importance',ascending=False)
plt.figure(figsize = (20,8))
plt.title('Features importance',fontsize=14)
s = sns.barplot(x='Feature',y='Feature importance',data=tmp_rf[:30])
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.show()  

# Compare the different model results

In [91]:
feature1 = tmp_cat[:30]["Feature"] 
feature2 = tmp_xgboost[:30]["Feature"]
feature3 = tmp_rf[:30]["Feature"]

In [92]:
featuresranking = pd.DataFrame()
featuresranking['feature1'] = feature1
featuresranking['feature2'] = feature2
featuresranking['feature3'] = feature3

# We can see the three models are working almost all very good, and the top features are nearly the same, especially the top 6 variables. However, they have slightly different results in the following features

In [93]:
featuresranking[:30] 

# We can see the top three variables actually have high corrlations between each other, howerver, as I mentioned, the tree and boosting algorithm are nonparametric models, they do not have assumptions such as the regression model, so they pick up those top features however they might have higly correlations within each other. 

In [104]:
sns.heatmap(train[['OverallQual','total_area','totalsf', 'OverallCond','YearBuilt']].corr(),cmap="YlGnBu", annot=True)

# Compare the RMSE results of the different models

# Catboost performs best amomg the three models on this dataset

In [105]:
print(cbresult['validation']['RMSE'][-1])
print(tempresulst[-1])
print(rmse_rf[-1])